In [59]:
# Import required libraries
from torchvision.datasets import ImageFolder
from torchvision import transforms
import timm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4590, 0.4330, 0.3856], std=[0.2328, 0.2162, 0.2158])
    ])

In [35]:
#Load Data
trainset = ImageFolder('Split/train/', transform=transform)
#testset = ImageFolder('Split/test/', transform=transform)
valset = ImageFolder('Split/val/', transform=transform)

In [14]:
# Calculate mean and std for normalizing
mean = torch.zeros(3)
std = torch.zeros(3)
for images, _ in trainset:
    mean += images.mean(dim=(1,2))
    std += images.std(dim=(1,2))

mean /= len(trainset)
std /= len(trainset)

print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: tensor([-1.4658e-04,  8.6977e-05,  1.3116e-04])
Std: tensor([1.0001, 1.0002, 1.0000])


In [49]:
# DataLoader
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
val_loader = DataLoader(valset, batch_size=32, shuffle=True)

In [23]:
#import the model
model = timm.create_model('efficientnet_b2', pretrained=True)

#transform = timm.data.create_transform(
#    **timm.data.resolve_data_config(model.pretrained_cfg)
#)

In [50]:
model.to(device)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [54]:
# Freeze the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_features = model.classifier.in_features
model.classifier = torch.nn.Linear(num_features, 30)

# Set the requires_grad attribute of the new fully connected layer to True
model.classifier.requires_grad = True

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=0.03)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)


In [56]:
num_epochs = 1



for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    total = 0
    
    for inputs, labels in train_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
    
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

        # Calculate the number of correct predictions
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        total += labels.size(0)

    # Calculate the average training loss and accuracy
    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / total


    # Set the model to evaluation mode
    model.eval()

    val_loss = 0
    val_correct = 0
    total = 0

    # Disable gradient computation during validation
    with torch.no_grad():
        # Loop over the batches in the validation loader
        for inputs, labels in val_loader:
            # Move the inputs and labels to the GPU if available
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Update the validation loss
            val_loss += loss.item() * inputs.size(0)

            # Calculate the number of correct predictions
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()
            total += labels.size(0)

    # Calculate the average validation loss and accuracy
    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / total

    # Print the training and validation loss and accuracy for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1/1: Train Loss: 2.0252, Train Acc: 0.5164, Val Loss: 1.4733, Val Acc: 0.6408
